In [53]:
!pip install yfinance
!pip install yahooquery
from yahooquery import Screener
import yfinance as yf   
import pandas as pd
import numpy as np
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense,Dropout
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
import random 
from tensorflow.keras.models import load_model
import os
import shutil
from datetime import datetime


In [74]:
clmns = [
    "Open 12-11","Open 11-10","Open 10-9","Open 9-8","Open 8-7","Open 7-6","Open 6-5","Open 5-4","Open 4-3","Open 3-2","Open 2-1",
    "Close 12-11","Close 11-10","Close 10-9","Close 9-8","Close 8-7","Close 7-6","Close 6-5","Close 5-4","Close 4-3","Close 3-2","Close 2-1",
    "High 12-11","High 11-10","High 10-9","High 9-8","High 8-7","High 7-6","High 6-5","High 5-4","High 4-3","High 3-2","High 2-1",
    "Low 12-11","Low 11-10","Low 10-9","Low 9-8","Low 8-7","Low 7-6","Low 6-5","Low 5-4","Low 4-3","Low 3-2","Low 2-1",
    "AdjV 12-11","AdjV 11-10","AdjV 10-9","AdjV 9-8","AdjV 8-7","AdjV 7-6","AdjV 6-5","AdjV 5-4","AdjV 4-3","AdjV 3-2","AdjV 2-1",
    "Volume 12-11","Volume 11-10","Volume 10-9","Volume 9-8","Volume 8-7","Volume 7-6","Volume 6-5","Volume 5-4","Volume 4-3","Volume 3-2","Volume 2-1",
    "suggestion"
]

def work_with_dir():
  if os.path.exists("/content/data/"):
    shutil.rmtree("/content/data/", ignore_errors=True)
    print("removed")
    os.mkdir("/content/data/")

  if not os.path.exists("/content/data/"):
    os.mkdir("/content/data/")
  
  if not os.path.exists("/content/extracted/"):
    os.mkdir("/content/extracted/")

def read_txt_list():
  with open("yahoo_stocklist.txt","r")as f:
    lines = f.readlines()
    nlines = []
    for line in lines:
       nlines.append(line.strip())
    return nlines
def read_syms_from_txt():
  with open("syms.txt", "r") as f:
    lines = f.readlines()
    f.close()
  lst = []
  for line in lines:
    line = line.strip()
    lst.append(line)
  symbols = lst
  return symbols
def get_crypto_syms():
   # 'all_cryptocurrencies_au','all_cryptocurrencies_ca','all_cryptocurrencies_eu','all_cryptocurrencies_gb','all_cryptocurrencies_in',
   screens = [
       'all_cryptocurrencies_us', 'all_cryptocurrencies_au', 'all_cryptocurrencies_ca', 'all_cryptocurrencies_eu', 'all_cryptocurrencies_gb', 'all_cryptocurrencies_in', ]
   s = Screener()
   symbols = []
   for i in screens:
      data = s.get_screeners(i, count=250)
      dicts = data[i]['quotes']
      syms = [d['symbol'] for d in dicts]
      for sym in syms:
        symbols.append(sym)
   # print(len(symbols))
   # pieces = 15
   # new_arrays = np.array_split(symbols, pieces)
   return symbols
def spliting(data):
  X = data.drop(["yes","no"], axis=1)
  y = data[["yes","no"]]
  xTrain, xTest, yTrain, yTest = train_test_split(X, y, test_size=0.1)
  print(xTrain.shape, end=" ")
  print(yTrain.shape)
  print(xTest.shape, end=" ")
  print(yTest.shape)
  return xTrain, xTest, yTrain, yTest
def scaler(row):
    scaler = MinMaxScaler(feature_range=(-1, 1))
    row = scaler.fit_transform(row)
    return row
def process(data):
    data = data.dropna()
    row = []
    data.drop("Date",axis=1,inplace=True)
    data = np.array(data)
    llst = [0, 1, 2, 3, 4, 5]
    for i in range(12, data.shape[0]):
        grow = []
        srow = []

        for j in llst:
           srow.append([
               data[i-1][j] - data[i-2][j],
               data[i-2][j] - data[i-3][j],
               data[i-3][j] - data[i-4][j],
               data[i-4][j] - data[i-5][j],
               data[i-5][j] - data[i-6][j],
               data[i-6][j] - data[i-7][j],
               data[i-7][j] - data[i-8][j],
               data[i-8][j] - data[i-9][j],
               data[i-9][j] - data[i-10][j],
               data[i-10][j] - data[i-11][j],
               data[i-11][j] - data[i-12][j]
           ])

        for lst in srow:
            mm = np.array(lst)
            mm = np.reshape(mm, (-1, 1))
            grow.append(scaler(mm))

        sugg = "no"
        if data[i][3] > data[i][0]:
            sugg = "yes"

        arr = np.array(grow).flatten()
        arr = np.append(arr, sugg)
        row.append(arr)


    grow = []
    srow = []
    llst = []
    data = []
    arr = []
    mm = []

    return np.array(row)

In [86]:
def download_data():

  symbols = get_crypto_syms()
  #symbols = read_txt_list()
  #symbols = read_syms_from_txt()

  work_with_dir()
  for symbol in symbols[:6]:
    try:
        data = yf.download(symbol, period="max",interval="1mo", progress=False,show_errors=False)
        if data.empty:
           pass
        else:
           if data.shape[0] > 12:
             data.to_csv(f"/content/data/{symbol}.csv")
             
    except:
       print("Error!")
  print(f"Files in data : {len(os.listdir('/content/data/'))}")

def extract_data():
  pd.options.mode.chained_assignment = None
  data = []
  unattached_dfs = []
  files = os.listdir("/content/data/")
  print("Processing File:")
  for file in files:
     print(f" {files.index(file)+1}",end=" ")
     address = f"/content/data/{file}"
     unattached_dfs.append(process(pd.read_csv(address)))
  print("")
  data = np.array(unattached_dfs[0])
  for i in unattached_dfs[1:]:
        data = np.append(data, np.array(i), axis=0)
        
  unattached_dfs = []
  
  data = pd.DataFrame(data, columns=clmns)
  sugg = data["suggestion"]
  data.drop("suggestion",axis=1,inplace=True)
  sugg = pd.get_dummies(sugg)
  data = pd.concat([data,sugg],axis=1)
  data = data.astype(float)

  print(data.shape)

  now = datetime.now().strftime("%H%M%S")

  data.to_csv(f"/content/extracted/{now}.csv")  

def get_xy(): 
    files = os.listdir("/content/extracted/")
    address = f"/content/extracted/{files[-1]}"
    data = pd.read_csv(address)
    xTrain,xTest,yTrain,yTest = spliting(data)
    data = []
    return xTrain,xTest,yTrain,yTest

In [85]:
download_data()
extract_data()
xTrain,xTest,yTrain,yTest = get_xy()


removed
Files in data : 6
Processing File:
 1  2  3  4  5  6 
(381, 68)
(342, 67) (342, 2)
(39, 67) (39, 2)


In [73]:
model = Sequential()



model.add(Dense(1024, activation='relu', input_shape=(xTrain.shape[1],)))
model.add(Dense(750, activation='relu'))
model.add(Dense(750, activation='relu'))
model.add(Dense(750, activation='relu'))
model.add(Dense(750, activation='relu'))
model.add(Dense(750, activation='relu'))



model.add(Dense(2, activation='sigmoid'))
model.compile(optimizer="adam", loss='binary_crossentropy',metrics=['accuracy'])
model.summary()

NameError: ignored

In [28]:
model.fit(xTrain,yTrain,epochs=35,batch_size=4096,validation_data=(xTest,yTest),shuffle=False)

Epoch 1/35
30/30 [==============================] - 41s 1s/step - loss: 0.6900 - accuracy: 0.5234 - val_loss: 0.6833 - val_accuracy: 0.5418
Epoch 2/35
30/30 [==============================] - 40s 1s/step - loss: 0.6752 - accuracy: 0.5586 - val_loss: 0.6836 - val_accuracy: 0.5552
Epoch 3/35
30/30 [==============================] - 40s 1s/step - loss: 0.6604 - accuracy: 0.5819 - val_loss: 0.6836 - val_accuracy: 0.5653
Epoch 4/35
30/30 [==============================] - 40s 1s/step - loss: 0.6542 - accuracy: 0.5934 - val_loss: 0.6794 - val_accuracy: 0.5716
Epoch 5/35
30/30 [==============================] - 40s 1s/step - loss: 0.6384 - accuracy: 0.6131 - val_loss: 0.6884 - val_accuracy: 0.5768
Epoch 6/35
30/30 [==============================] - 40s 1s/step - loss: 0.6225 - accuracy: 0.6304 - val_loss: 0.7105 - val_accuracy: 0.5787
Epoch 7/35
30/30 [==============================] - 40s 1s/step - loss: 0.6414 - accuracy: 0.6092 - val_loss: 0.7430 - val_accuracy: 0.5736
Epoch 8/35
30/30 [==

KeyboardInterrupt: ignored

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive
